# 04 — Business Impact, Threshold Tuning & Recommendations

Fraud detection is not just a technical classification task —
it directly impacts financial losses and customer experience.

Goal:
- Apply threshold tuning to maximize financial net benefit
- Provide actionable insights for fraud risk teams


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# Load dataset
df = pd.read_csv("../data/processed/onlinefraud_clean.csv")
X = df.drop(columns=["isFraud"])
y = df["isFraud"]

# SAME split to reuse consistency
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Reload trained model (you may train again if needed)
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(
    n_estimators=200, max_depth=8, class_weight="balanced", random_state=42)
model_rf.fit(X_train, y_train)

y_probs = model_rf.predict_proba(X_test)[:,1]


## Cost/Benefit Assumptions:
- Prevented fraud saves: ₹10,000 per transaction
- Reviewing a legitimate flagged case: cost ₹200


In [ ]:
thresholds = np.linspace(0,1,100)
profits = []

for t in thresholds:
    preds = (y_probs >= t).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()

    revenue = tp * 10000        # caught fraud saves money
    review_cost = fp * 200      # each false alert wastes resources
    profits.append(revenue - review_cost)

plt.figure(figsize=(8,5))
plt.plot(thresholds, profits)
plt.xlabel("Threshold")
plt.ylabel("Net Profit (₹)")
plt.title("Profit Optimization by Threshold")
plt.show()

best_t = thresholds[np.argmax(profits)]
best_profit = max(profits)
best_t, best_profit


## 🧠 Business Recommendations

- Focus monitoring rules on **TRANSFER** and **CASH_OUT** transactions
- Use **threshold = {best_t:.2f}** for optimal profitability
- Implement **merchant-origin behavior checks**
- Prioritize first-time merchants for manual review
- Deploy targeted advisor compliance training

📈 Operational impact:
- Improves fraud catch rate
- Reduces unnecessary friction for real users
- Maximizes financial protection with efficient review cost
